# Transformation des données de pandémies - Partie 1: Nettoyage

Ce notebook réalise le nettoyage des données extraites pour les préparer à la transformation.

## 1. Configuration initiale

Importation des bibliothèques nécessaires et chargement des données extraites.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

# Configuration de la visualisation
%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_intermediaires = chemin_base / "donnees" / "intermediaires"
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Affichage des chemins
print(f"Dossier des données intermédiaires: {dossier_donnees_intermediaires}")
print(f"Dossier des données transformées: {dossier_donnees_transformees}")

In [ ]:
# Chargement des données extraites
print("Chargement des données extraites...")

donnees = {}

try:
    # Chargement des données COVID-19
    fichier_covid19 = dossier_donnees_intermediaires / "covid19_extrait.csv"
    if fichier_covid19.exists():
        donnees["covid19"] = pd.read_csv(fichier_covid19)
        print(f"Données COVID-19 chargées: {donnees['covid19'].shape[0]} lignes, {donnees['covid19'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger les données COVID-19: fichier non trouvé")
    
    # Chargement des données MPOX
    fichier_mpox = dossier_donnees_intermediaires / "mpox_extrait.csv"
    if fichier_mpox.exists():
        donnees["mpox"] = pd.read_csv(fichier_mpox)
        print(f"Données MPOX chargées: {donnees['mpox'].shape[0]} lignes, {donnees['mpox'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger les données MPOX: fichier non trouvé")
    
    # Chargement des données Worldometer
    fichier_worldometer = dossier_donnees_intermediaires / "worldometer_extrait.csv"
    if fichier_worldometer.exists():
        donnees["worldometer"] = pd.read_csv(fichier_worldometer)
        print(f"Données Worldometer chargées: {donnees['worldometer'].shape[0]} lignes, {donnees['worldometer'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger les données Worldometer: fichier non trouvé")
    
    print("\nChargement des données terminé avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement des données: {e}")

## 2. Nettoyage des données COVID-19

Nettoyage des données COVID-19 pour les préparer à la transformation.

In [ ]:
# Nettoyage des données COVID-19
if "covid19" in donnees:
    print("Nettoyage des données COVID-19...")
    
    # Copie des données pour éviter de modifier les originales
    covid19_clean = donnees["covid19"].copy()
    
    # Conversion de la colonne Date en datetime
    covid19_clean["Date"] = pd.to_datetime(covid19_clean["Date"])
    
    # Standardisation des noms de colonnes
    covid19_clean = covid19_clean.rename(columns={
        "Date": "date",
        "Country/Region": "pays",
        "Province/State": "region",
        "Confirmed": "cas_confirmes",
        "Deaths": "deces",
        "Recovered": "guerisons",
        "Active": "cas_actifs"
    })
    
    # Ajout d'une colonne pour le type de pandémie
    covid19_clean["type_pandemie"] = "COVID-19"
    
    # Gestion des valeurs manquantes
    for col in ["cas_confirmes", "deces", "guerisons", "cas_actifs"]:
        covid19_clean[col] = covid19_clean[col].fillna(0).astype(int)
    
    # Affichage des informations après nettoyage
    print(f"Dimensions après nettoyage: {covid19_clean.shape[0]} lignes, {covid19_clean.shape[1]} colonnes")
    print("Colonnes après nettoyage:")
    for col in covid19_clean.columns:
        print(f"- {col}")
    print("\nAperçu des données après nettoyage:")
    display(covid19_clean.head())
    
    # Vérification des valeurs manquantes après nettoyage
    valeurs_manquantes = covid19_clean.isnull().sum()
    print("\nValeurs manquantes après nettoyage:")
    display(valeurs_manquantes[valeurs_manquantes > 0])
    
    # Sauvegarde des données nettoyées
    donnees["covid19_clean"] = covid19_clean
    print("Nettoyage des données COVID-19 terminé avec succès!")

## 3. Nettoyage des données MPOX

Nettoyage des données MPOX pour les préparer à la transformation.

In [ ]:
# Nettoyage des données MPOX
if "mpox" in donnees:
    print("Nettoyage des données MPOX...")
    
    # Copie des données pour éviter de modifier les originales
    mpox_clean = donnees["mpox"].copy()
    
    # Conversion de la colonne date en datetime
    mpox_clean["date"] = pd.to_datetime(mpox_clean["date"])
    
    # Standardisation des noms de colonnes
    mpox_clean = mpox_clean.rename(columns={
        "location": "pays",
        "total_cases": "cas_confirmes",
        "total_deaths": "deces",
        "new_cases": "nouveaux_cas",
        "new_deaths": "nouveaux_deces"
    })
    
    # Ajout d'une colonne pour le type de pandémie
    mpox_clean["type_pandemie"] = "MPOX"
    
    # Ajout de colonnes manquantes pour uniformiser avec COVID-19
    mpox_clean["region"] = np.nan
    mpox_clean["guerisons"] = np.nan
    mpox_clean["cas_actifs"] = np.nan
    
    # Sélection des colonnes pertinentes
    mpox_clean = mpox_clean[[
        "date", "pays", "region", "cas_confirmes", "deces", 
        "guerisons", "cas_actifs", "type_pandemie"
    ]]
    
    # Gestion des valeurs manquantes
    for col in ["cas_confirmes", "deces"]:
        mpox_clean[col] = mpox_clean[col].fillna(0)
    
    # Conversion des types de données
    mpox_clean["cas_confirmes"] = mpox_clean["cas_confirmes"].astype(float).astype(int)
    mpox_clean["deces"] = mpox_clean["deces"].astype(float).astype(int)
    
    # Affichage des informations après nettoyage
    print(f"Dimensions après nettoyage: {mpox_clean.shape[0]} lignes, {mpox_clean.shape[1]} colonnes")
    print("Colonnes après nettoyage:")
    for col in mpox_clean.columns:
        print(f"- {col}")
    print("\nAperçu des données après nettoyage:")
    display(mpox_clean.head())
    
    # Vérification des valeurs manquantes après nettoyage
    valeurs_manquantes = mpox_clean.isnull().sum()
    print("\nValeurs manquantes après nettoyage:")
    display(valeurs_manquantes[valeurs_manquantes > 0])
    
    # Sauvegarde des données nettoyées
    donnees["mpox_clean"] = mpox_clean
    print("Nettoyage des données MPOX terminé avec succès!")

## 4. Nettoyage des données Worldometer

Nettoyage des données Worldometer pour les préparer à la transformation.

In [ ]:
# Nettoyage des données Worldometer
if "worldometer" in donnees:
    print("Nettoyage des données Worldometer...")
    
    # Copie des données pour éviter de modifier les originales
    worldometer_clean = donnees["worldometer"].copy()
    
    # Conversion de la colonne date en datetime
    worldometer_clean["date"] = pd.to_datetime(worldometer_clean["date"])
    
    # Standardisation des noms de colonnes
    worldometer_clean = worldometer_clean.rename(columns={
        "country": "pays",
        "cumulative_total_cases": "cas_confirmes",
        "cumulative_total_deaths": "deces",
        "daily_new_cases": "nouveaux_cas",
        "daily_new_deaths": "nouveaux_deces"
    })
    
    # Ajout d'une colonne pour le type de pandémie
    worldometer_clean["type_pandemie"] = "COVID-19"
    
    # Ajout de colonnes manquantes pour uniformiser avec COVID-19
    worldometer_clean["region"] = np.nan
    worldometer_clean["guerisons"] = np.nan
    worldometer_clean["cas_actifs"] = np.nan
    
    # Sélection des colonnes pertinentes
    worldometer_clean = worldometer_clean[[
        "date", "pays", "region", "cas_confirmes", "deces", 
        "guerisons", "cas_actifs", "type_pandemie"
    ]]
    
    # Gestion des valeurs manquantes
    for col in ["cas_confirmes", "deces"]:
        worldometer_clean[col] = worldometer_clean[col].fillna(0).astype(int)
    
    # Affichage des informations après nettoyage
    print(f"Dimensions après nettoyage: {worldometer_clean.shape[0]} lignes, {worldometer_clean.shape[1]} colonnes")
    print("Colonnes après nettoyage:")
    for col in worldometer_clean.columns:
        print(f"- {col}")
    print("\nAperçu des données après nettoyage:")
    display(worldometer_clean.head())
    
    # Vérification des valeurs manquantes après nettoyage
    valeurs_manquantes = worldometer_clean.isnull().sum()
    print("\nValeurs manquantes après nettoyage:")
    display(valeurs_manquantes[valeurs_manquantes > 0])
    
    # Sauvegarde des données nettoyées
    donnees["worldometer_clean"] = worldometer_clean
    print("Nettoyage des données Worldometer terminé avec succès!")

## 5. Sauvegarde des données nettoyées

Sauvegarde des données nettoyées pour l'étape suivante de transformation.

In [ ]:
# Sauvegarde des données nettoyées
print("Sauvegarde des données nettoyées...")

try:
    # Création du dossier de données transformées s'il n'existe pas
    if not dossier_donnees_transformees.exists():
        dossier_donnees_transformees.mkdir(parents=True, exist_ok=True)
        print(f"Dossier créé: {dossier_donnees_transformees}")
    
    # Sauvegarde des données COVID-19 nettoyées
    if "covid19_clean" in donnees:
        chemin_fichier = dossier_donnees_transformees / "covid19_nettoye.csv"
        donnees["covid19_clean"].to_csv(chemin_fichier, index=False)
        print(f"Données COVID-19 nettoyées sauvegardées: {chemin_fichier}")
    
    # Sauvegarde des données MPOX nettoyées
    if "mpox_clean" in donnees:
        chemin_fichier = dossier_donnees_transformees / "mpox_nettoye.csv"
        donnees["mpox_clean"].to_csv(chemin_fichier, index=False)
        print(f"Données MPOX nettoyées sauvegardées: {chemin_fichier}")
    
    # Sauvegarde des données Worldometer nettoyées
    if "worldometer_clean" in donnees:
        chemin_fichier = dossier_donnees_transformees / "worldometer_nettoye.csv"
        donnees["worldometer_clean"].to_csv(chemin_fichier, index=False)
        print(f"Données Worldometer nettoyées sauvegardées: {chemin_fichier}")
    
    print("\nSauvegarde des données nettoyées terminée avec succès!")
except Exception as e:
    print(f"Erreur lors de la sauvegarde des données nettoyées: {e}")

## Conclusion

Dans ce notebook, nous avons réalisé le nettoyage des données extraites pour les préparer à la transformation. Les principales opérations de nettoyage ont été :

1. Conversion des dates au format datetime
2. Standardisation des noms de colonnes
3. Ajout d'une colonne pour le type de pandémie
4. Gestion des valeurs manquantes
5. Conversion des types de données

Les données nettoyées ont été sauvegardées dans des fichiers CSV pour l'étape suivante de transformation : la suppression des doublons et l'agrégation des données.